In [1]:
!pip install tensorflow[and-cuda]
!pip install matplotlib
!pip install tables
!pip install pillow
!pip install opencv-python

ERROR: Could not find a version that satisfies the requirement tensorflow[and-cuda] (from versions: none)
ERROR: No matching distribution found for tensorflow[and-cuda]


   ---------------------------------------- 0.0/8.0 MB ? eta -:--:--
   ------------------- -------------------- 3.9/8.0 MB 20.3 MB/s eta 0:00:01
   ------------------------------------- -- 7.6/8.0 MB 18.6 MB/s eta 0:00:01
   ---------------------------------------- 8.0/8.0 MB 18.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.2 MB ? eta -:--:--
   ---------------------------------------- 2.2/2.2 MB 30.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/12.6 MB ? eta -:--:--
   --------------------- ------------------ 6.8/12.6 MB 32.2 MB/s eta 0:00:01
   ---------------------------------------- 12.6/12.6 MB 30.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.6 MB ? eta -:--:--
   ---------------------------------------- 2.6/2.6 MB 29.1 MB/s eta 0:00:00
     ---------------------------------------- 0.0/4.8 MB ? eta -:--:--
     ---------------------- ----------------- 2.6/4.8 MB 15.3 MB/s eta 0:00:01
     --------------------------

  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> [34 lines of output]
      * Using Python 3.13.1 (tags/v3.13.1:0671451, Dec  3 2024, 19:06:28) [MSC v.1942 64 bit (AMD64)]
      * Found cython 3.0.11
      * USE_PKGCONFIG: False
      Traceback (most recent call last):
        File "c:\Users\25wan\Documents\GitHub\greek-letter-cnn\.venv\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 353, in <module>
          main()
          ~~~~^^
        File "c:\Users\25wan\Documents\GitHub\greek-letter-cnn\.venv\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 335, in main
          json_out['return_val'] = hook(**hook_input['kwargs'])
                                   ~~~~^^^^^^^^^^^^^^^^^^^^^^^^
        File "c:\Users\25wan\Documents\GitHub\greek-letter-cnn\.venv\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 118, in ge

   ---------------------------------------- 0.0/38.8 MB ? eta -:--:--
   --- ------------------------------------ 3.1/38.8 MB 17.1 MB/s eta 0:00:03
   ------- -------------------------------- 7.6/38.8 MB 19.8 MB/s eta 0:00:02
   ------------ --------------------------- 12.1/38.8 MB 20.6 MB/s eta 0:00:02
   ----------------- ---------------------- 16.5/38.8 MB 21.0 MB/s eta 0:00:02
   ---------------------- ----------------- 21.8/38.8 MB 21.9 MB/s eta 0:00:01
   ----------------------------- ---------- 28.6/38.8 MB 23.9 MB/s eta 0:00:01
   ------------------------------------ --- 35.9/38.8 MB 25.8 MB/s eta 0:00:01
   ---------------------------------------- 38.8/38.8 MB 25.6 MB/s eta 0:00:00


In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import os
from PIL import Image 
import cv2
from random import shuffle
import glob
import tables
import cv2  
from math import ceil
print("Num GPUs Available: ", len(tf.config.list_physical_devices("GPU")))


Num GPUs Available:  0


In [38]:
hdf5_path = "./data/greekdata.hdf5"
train_path = "./out/*/*.png"

In [39]:
addrs = glob.glob(train_path)
labels = [addr.split("\\")[1] for addr in addrs]
unique = set(labels)
uniqueList = list(unique)
for i in range(len(labels)):
    labels[i] = uniqueList.index(labels[i])

c = list(zip(addrs, labels))
shuffle(c)
addrs, labels = zip(*c)
train_addrs = addrs[0:int(0.6*len(addrs))]
train_labels = labels[0:int(0.6*len(labels))]

val_addrs = addrs[int(0.6*len(addrs)):int(0.8*len(addrs))]
val_labels = labels[int(0.6*len(addrs)):int(0.8*len(addrs))]

test_addrs = addrs[int(0.8*len(addrs)):]
test_labels = labels[int(0.8*len(labels)):]
print("train size:",len(train_addrs))
print("val size:",len(val_addrs))
print("test size:",len(test_addrs))
print(labels[:5])

train size: 99963
val size: 33321
test size: 33321
(193, 150, 162, 6, 42)


In [40]:
data_order = "tf"
img_dtype = tables.UInt8Atom() 


data_shape = (0, 32, 32, 3)

hdf5_file = tables.open_file(hdf5_path, mode="w")
try:
    train_storage = hdf5_file.create_earray(hdf5_file.root, "train_img", img_dtype, shape=data_shape)
    val_storage = hdf5_file.create_earray(hdf5_file.root, "val_img", img_dtype, shape=data_shape)
    test_storage = hdf5_file.create_earray(hdf5_file.root, "test_img", img_dtype, shape=data_shape)
    
    mean_storage = hdf5_file.create_earray(hdf5_file.root, "train_mean", img_dtype, shape=data_shape)
    
    hdf5_file.create_array(hdf5_file.root, "train_labels", train_labels)
    hdf5_file.create_array(hdf5_file.root, "val_labels", val_labels)
    hdf5_file.create_array(hdf5_file.root, "test_labels", test_labels)
    
    mean = np.zeros(data_shape[1:], np.float32)
    
    for i in range(len(train_addrs)):
        if i % 10 == 0 and i > 1:
            print("Train data: {}/{}".format(i, len(train_addrs)))
        addr = train_addrs[i]
        img = cv2.imread(addr)
        img = cv2.resize(img, (32, 32), interpolation=cv2.INTER_CUBIC)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
        train_storage.append(img[None])
        mean += img / float(len(train_labels))
    
    for i in range(len(val_addrs)): 
        if i % 10 == 0 and i > 1:
            print ("Validation data: {}/{}".format(i, len(val_addrs)))
    

        addr = val_addrs[i]
        img = cv2.imread(addr)
        img = cv2.resize(img, (32, 32), interpolation=cv2.INTER_CUBIC)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        val_storage.append(img[None])
    
    for i in range(len(test_addrs)):
        if i % 10 == 0 and i > 1:
            print ("Test data: {}/{}".format(i, len(test_addrs)))

        addr = test_addrs[i]
        img = cv2.imread(addr)
        img = cv2.resize(img, (32, 32), interpolation=cv2.INTER_CUBIC)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        test_storage.append(img[None])
    
    mean_storage.append(mean[None])
    print("HDF5 Done")
finally: 
    print("In Finally")
    hdf5_file.close()

Train data: 10/99963
Train data: 20/99963
Train data: 30/99963
Train data: 40/99963
Train data: 50/99963
Train data: 60/99963
Train data: 70/99963
Train data: 80/99963
Train data: 90/99963
Train data: 100/99963
Train data: 110/99963
Train data: 120/99963
Train data: 130/99963
Train data: 140/99963
Train data: 150/99963
Train data: 160/99963
Train data: 170/99963
Train data: 180/99963
Train data: 190/99963
Train data: 200/99963
Train data: 210/99963
Train data: 220/99963
Train data: 230/99963
Train data: 240/99963
Train data: 250/99963
Train data: 260/99963
Train data: 270/99963
Train data: 280/99963
Train data: 290/99963
Train data: 300/99963
Train data: 310/99963
Train data: 320/99963
Train data: 330/99963
Train data: 340/99963
Train data: 350/99963
Train data: 360/99963
Train data: 370/99963
Train data: 380/99963
Train data: 390/99963
Train data: 400/99963
Train data: 410/99963
Train data: 420/99963
Train data: 430/99963
Train data: 440/99963
Train data: 450/99963
Train data: 460/999

In [41]:
subtract_mean = True
batch_size = 50
nb_class = 314

In [44]:
hdf5_file = tables.open_file(hdf5_path, mode="r")
if subtract_mean:
    mm = hdf5_file.root.train_mean[0]
    mm = mm[np.newaxis, ...]

train_data = np.array(hdf5_file.root.train_img)
train_label = np.array(hdf5_file.root.train_labels)

test_data = np.array(hdf5_file.root.test_img)
test_label = np.array(hdf5_file.root.test_labels)

val_data = np.array(hdf5_file.root.val_img)
val_label = np.array(hdf5_file.root.val_labels)

print("train data:",train_data.shape," train_label",train_label.shape)
print("test_data:",test_data.shape," test_label:",test_label.shape)
print("val_data:",val_data.shape," val_label:",val_label.shape)

train data: (99963, 32, 32, 3)  train_label (99963,)
test_data: (33321, 32, 32, 3)  test_label: (33321,)
val_data: (33321, 32, 32, 3)  val_label: (33321,)


In [45]:
from tensorflow.keras.utils import to_categorical
num_classes = len(np.unique(train_label))
print("num_classes:",num_classes)
print(train_label[:5])
train_label = to_categorical(train_label)
test_label = to_categorical(test_label, num_classes)
val_label = to_categorical(val_label, num_classes)

# print shape of training set
print("num_classes:", num_classes)

# print number of training, validation, and test images
print(train_label.shape, "train samples")
print(test_label.shape, "test samples")
print(val_label.shape, "validation samples")

num_classes: 314
[193 150 162   6  42]
num_classes: 314
(99963, 314) train samples
(33321, 314) test samples
(33321, 314) validation samples


In [52]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Conv2D(filters=16, kernel_size=2, padding='same', activation='relu', 
                        input_shape=(32, 32, 3)))
model.add(tf.keras.layers.MaxPooling2D(pool_size=2))
model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=2, padding='same', activation='tanh'))
model.add(tf.keras.layers.MaxPooling2D(pool_size=2))
# model.add(tf.keras.layers.Conv2D(filters=64, kernel_size=2, padding='same', activation='tanh'))
# model.add(tf.keras.layers.MaxPooling2D(pool_size=2))
# model.add(tf.keras.layers.Dropout(0.3))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(512, activation='relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(314, activation='softmax'))
model.summary()


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 32, 32, 16)        208       
                                                                 
 max_pooling2d_4 (MaxPoolin  (None, 16, 16, 16)        0         
 g2D)                                                            
                                                                 
 conv2d_5 (Conv2D)           (None, 16, 16, 32)        2080      
                                                                 
 max_pooling2d_5 (MaxPoolin  (None, 8, 8, 32)          0         
 g2D)                                                            
                                                                 
 flatten_2 (Flatten)         (None, 2048)              0         
                                                                 
 dense_4 (Dense)             (None, 512)              

In [53]:
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', 
                  metrics=['accuracy'])

In [ ]:
from keras.callbacks import ModelCheckpoint   

checkpointer = ModelCheckpoint(filepath='model.weights.best_200.hdf5', verbose=1, 
                               save_best_only=True,monitor='val_accuracy', mode = "max")
hist = model.fit(train_data, train_label, batch_size=None, epochs=200,
          validation_data=(val_data, val_label),callbacks=[checkpointer], 
          verbose=1, shuffle=True)

Epoch 1/200
3121/3124 [============================>.] - ETA: 0s - loss: 3.4675 - accuracy: 0.3061
Epoch 1: val_accuracy improved from -inf to 0.52967, saving model to model.weights.best_200.hdf5
3124/3124 [==============================] - 45s 14ms/step - loss: 3.4665 - accuracy: 0.3062 - val_loss: 2.2150 - val_accuracy: 0.5297
Epoch 2/200
   5/3124 [..............................] - ETA: 39s - loss: 2.9497 - accuracy: 0.4313

c:\Users\25wan\Documents\GitHub\greek-letter-cnn\.venv\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


3123/3124 [============================>.] - ETA: 0s - loss: 3.0231 - accuracy: 0.3868
Epoch 2: val_accuracy improved from 0.52967 to 0.53063, saving model to model.weights.best_200.hdf5
3124/3124 [==============================] - 44s 14ms/step - loss: 3.0231 - accuracy: 0.3868 - val_loss: 2.2345 - val_accuracy: 0.5306
Epoch 3/200
2562/3124 [=======================>......] - ETA: 7s - loss: 3.0209 - accuracy: 0.3918